Below is summary of data schema - 
![image](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
train_data_df = pd.read_csv('../input/application_train.csv')

# Label Encoding

In [19]:
from sklearn.preprocessing import LabelEncoder

In [31]:
def label_encoder(dfname):
    le = LabelEncoder()
    j = 0
    for column in dfname:
        if dfname[column].dtype == 'object':
            if len(list(dfname[column].unique())) <= 2:
                le.fit(dfname[column])
                dfname[column] = le.transform(dfname[column])
                j += 1
    print(str(j) + ' Columns encoded')

In [32]:
label_encoder(train_data_df)
train_data_df = pd.get_dummies(train_data_df)

3 Columns encoded


# For each table counting no. of entries for each loan and averaging out all entries to create one row per loan
1. Count the no. of entries in new table which has primary key SK_ID_PREV for each value of SK_ID_CURR which is the foreign key referring to a loan
2. take rest of columns and take average of all entries
3. append table identifier to prevent name clashes
4. Join the table with main table

# Previous loan applications data

In [15]:
prev_applications_df = pd.read_csv('../input/previous_application.csv')
label_encoder(prev_applications_df)
prev_applications_df = pd.get_dummies(prev_applications_df)

In [16]:
prev_applications_df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
previous_apps_count = prev_applications_df[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
prev_applications_df['SK_ID_PREV'] = prev_applications_df['SK_ID_CURR'].map(previous_apps_count['SK_ID_PREV'])
previous_apps_mean = prev_applications_df.groupby('SK_ID_CURR').mean()
previous_apps_mean.columns = ['prev_apps' + col for col in previous_apps_mean.columns]
train_data_df = train_data_df.merge(right=previous_apps_mean.reset_index(), how='left', on='SK_ID_CURR')

# Previous loan payments data

In [8]:
installments_df = pd.read_csv('../input/installments_payments.csv')
label_encoder(installments_df)
installments_df = pd.get_dummies()

In [9]:
installments_df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [10]:
installments_count = installments_df[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
installments_df['SK_ID_PREV'] = installments_df['SK_ID_CURR'].map(installments_count['SK_ID_PREV'])
installments_mean = installments_df.groupby('SK_ID_CURR').mean()
installments_mean.columns = ['installment_' + col for col in installments_mean.columns]
train_data_df = train_data_df.merge(right=installments_mean.reset_index(), how='left', on='SK_ID_CURR')

# Monthly balance data of loans previously held

In [11]:
previous_loans_df = pd.read_csv('../input/POS_CASH_balance.csv')
label_encoder(previous_loans_df)
previous_loans_df = pd.get_dummies(previous_loans_df)

In [12]:
previous_loans_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [ ]:
previous_loans_count = previous_loans_df[['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').count()
previous_loans_df['SK_ID_PREV'] = previous_loans_df['SK_ID_CURR'].map(previous_apps_count['SK_ID_PREV'])
previous_loans_mean = previous_loans_df.groupby('SK_ID_CURR').mean()
previous_loans_mean.columns = ['prev_loans_' + col for col in previous_loans_mean.columns]
train_data_df = train_data_df.merge(right=previous_loans_mean.reset_index(), how='left', on='SK_ID_CURR')

# Credit card balance data

In [3]:
credit_card_df = pd.read_csv('../input/credit_card_balance.csv')
label_encoder(credit_card_df)
credit_card_df = pd.get_dummies(credit_card_df)

In [5]:
credit_card_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [6]:
credit_card_count = credit_card_df[['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').count()
credit_card_df['SK_ID_PREV'] = credit_card_df['SK_ID_CURR'].map(credit_card_count['SK_ID_PREV'])
credit_card_mean = credit_card_df.groupby('SK_ID_CURR').mean()
credit_card_mean.columns = ['card_'+ col for col in credit_card_mean.columns]
train_data_df = train_data_df.merge(right=credit_card_mean.reset_index(), how='left',on='SK_ID_CURR')

# Applicant's credit bureau history

In [21]:
bureau_history_df = pd.read_csv('../input/bureau.csv')
label_encoder(bureau_history_df)
bureau_history_df = pd.get_dummies(bureau_history_df)

0 Columns encoded


In [22]:
bureau_history_df.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [12]:
bureau_history_count =bureau_history_df[['SK_ID_CURR','SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
bureau_history_df['SK_ID_BUREAU'] = bureau_history_df['SK_ID_CURR'].map(bureau_history_count['SK_ID_BUREAU'])
bureau_history_mean = bureau_history_df.groupby('SK_ID_CURR').mean()
bureau_history_mean.columns = ['bureau_' + col for col in bureau_history_mean.columns]
train_data_df = train_data_df.merge(right=bureau_history_mean.reset_index(), how='left', on='SK_ID_CURR')

# Imputing Missing Values

In [35]:
from sklearn.impute import SimpleImputer

In [34]:
X = train_data_df.drop('TARGET',axis=1)
list_of_features = X.columns
Y = train_data_df['TARGET']

imputer = SimpleImputer()

X = pd.DataFrame(imputer.fit_transform(X))
X.columns = list_of_features

In [36]:
Y.to_csv('../input/final_target.csv')

In [37]:
X.to_csv('../input/final_data.csv')